In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
cwd = os.getcwd()

def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    print('')
    print('')
    return mae, rmse

In [3]:
sl = pd.read_csv('dataset/SCED (5m).csv')
sl['sced_time_stamp_local'] = pd.to_datetime(sl['sced_time_stamp_local'])
sl.set_index('sced_time_stamp_local', inplace=True)
sl = sl.resample('h').mean()
date_range = pd.date_range(start=sl.index.min(), end=sl.index.max(), freq='h')
sl = sl[~sl.index.duplicated(keep='first')]
sl = sl.reindex(date_range, fill_value=np.nan)
sl.interpolate(method='time', inplace=True)


In [4]:
dap = pd.read_csv('dataset/DAP (1hr).csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='h')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

/var/folders/kv/ldp2ch2s1md4v31v75w2dt2c0000gn/T/ipykernel_72557/1171035437.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dap['timestamp'] = pd.to_datetime(dap['timestamp'])


In [5]:
forecasted_tlf = pd.read_csv('dataset/FORECASTED_TLF (15m).csv')
forecasted_tlf['datetime'] = pd.to_datetime(forecasted_tlf['datetime'])
forecasted_tlf.set_index('datetime', inplace=True)
date_range = pd.date_range(start=forecasted_tlf.index.min(), end='2024-01-01 23:55:00', freq='h')
forecasted_tlf = forecasted_tlf[~forecasted_tlf.index.duplicated(keep='first')]
forecasted_tlf = forecasted_tlf.reindex(date_range, fill_value=np.nan)
forecasted_tlf.interpolate(method='time', inplace=True)


In [6]:
#prepare the data for training and testing
df = pd.concat([dap, sl, forecasted_tlf], axis=1)
df.columns = ['DAP', 'SCED', 'F_TLF']
df.dropna(inplace=True)

df

,DAP,SCED,F_TLF
2019-01-02 00:00:00,23.9250,26.216109,0.016499
2019-01-02 01:00:00,23.3140,25.671136,0.016441
2019-01-02 02:00:00,23.3475,24.970952,0.016411
2019-01-02 03:00:00,23.0595,24.787712,0.016441
2019-01-02 04:00:00,25.2672,24.887786,0.016534
...,...,...,...
2023-12-31 19:00:00,19.3715,14.422107,0.021356
2023-12-31 20:00:00,16.7739,12.597410,0.021367
2023-12-31 21:00:00,15.0035,10.170172,0.021383
2023-12-31 22:00:00,15.7699,9.872549,0.021408


In [7]:

## log transform
log_data = df.copy(deep=True)
log_data.loc[:,"SCED"] = np.log(df.loc[:,"SCED"] + 1 - min(df.loc[:,"SCED"]))
log_data.loc[:,"DAP"] = np.log(df.loc[:,"DAP"] + 1 - min(df.loc[:,"DAP"]))
log_data.loc[:,"F_TLF"] = np.log(df.loc[:,"F_TLF"] + 1 - min(df.loc[:,"F_TLF"]))

#Shift for prediction
df['DAP'] = df['DAP'].shift(-24)

# 3 years training, 1 year validation, 1 year testing
x_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:,:]
x_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:,:]
x_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:,:]

# Shift back for target variable
df['DAP'] = df['DAP'].shift(24)

#Change target variable to DAP
y_train_df_reg = log_data.loc[:'2021-12-31 23:55:00'].iloc[:, :1]
y_val_df_reg = log_data.loc['2022-01-01 00:00:00':'2022-12-31 23:55:00'].iloc[:, :1]
y_test_df_reg = log_data.loc['2023-01-01 00:00:00':].iloc[:, :1]

x_train_df_reg.reset_index(drop=True, inplace=True)
x_val_df_reg.reset_index(drop=True, inplace=True)
x_test_df_reg.reset_index(drop=True, inplace=True)

# Standardization
x_mean_reg, x_std_reg = x_train_df_reg.mean(), x_train_df_reg.std()
y_mean_reg, y_std_reg = y_train_df_reg.mean(), y_train_df_reg.std()

x_std_reg = x_std_reg +0.00001

x_train_reg = (x_train_df_reg - x_mean_reg)/x_std_reg
x_val_reg = (x_val_df_reg - x_mean_reg)/x_std_reg
x_test_reg = (x_test_df_reg - x_mean_reg)/x_std_reg

y_train_reg = (y_train_df_reg - y_mean_reg)/y_std_reg
y_val_reg = (y_val_df_reg - y_mean_reg)/y_std_reg
y_test_reg = (y_test_df_reg - y_mean_reg)/y_std_reg

# Shift the data for the lags
n_steps_in = 24
n_steps_out = 24

x_train_lstm = np.array([x_train_reg[i:i+n_steps_in] for i in range(0, x_train_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_train_lstm = np.array([y_train_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_train_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_val_lstm = np.array([x_val_reg[i:i+n_steps_in] for i in range(0, x_val_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_val_lstm = np.array([y_val_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_val_reg.shape[0]-n_steps_in-n_steps_out+1)])

x_test_lstm = np.array([x_test_reg[i:i+n_steps_in] for i in range(0, x_test_reg.shape[0]-n_steps_in-n_steps_out+1)])
y_test_lstm = np.array([y_test_reg[i+n_steps_in:i+n_steps_in+n_steps_out] for i in range(0, y_test_reg.shape[0]-n_steps_in-n_steps_out+1)])

print(x_train_lstm.shape,y_train_lstm.shape,x_val_lstm.shape,y_val_lstm.shape, x_test_lstm.shape,y_test_lstm.shape)

(26233, 24, 3) (26233, 24, 1) (8713, 24, 3) (8713, 24, 1) (8713, 24, 3) (8713, 24, 1)


In [8]:
y_train_lstm = y_train_lstm.squeeze(-1)
y_val_lstm = y_val_lstm.squeeze(-1)
y_test_lstm = y_test_lstm.squeeze(-1)

In [9]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a directory for saving the model if it doesn't exist
model_dir = os.path.join(os.getcwd(), 'saved_model')
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [10]:
# Define the PyTorch LSTM model (architecture mirrors your Keras model)
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc1   = nn.Linear(hidden_size, hidden_size)
        self.relu  = nn.ReLU()
        self.fc2   = nn.Linear(hidden_size, output_size)  # output_size equals n_steps_out (24)
        
    def forward(self, x):
        # x shape: (batch, seq_len, features)
        out, _ = self.lstm1(x)       # out shape: (batch, seq_len, hidden_size)
        out, _ = self.lstm2(out)     # still (batch, seq_len, hidden_size)
        out = out[:, -1, :]          # take output from the last time step -> (batch, hidden_size)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)          # final shape: (batch, output_size)
        return out

In [11]:
# Set hyperparameters (same as before)
n_neurons = 64
learning_rate = 0.0005
batch_size = 64
num_epochs = 50
patience = 10  # for early stopping

input_size = x_train_lstm.shape[2]  # number of features
output_size = y_train_lstm.shape[1]  # number of time steps out (24)

# Initialize model, loss function, and optimizer
model = LSTMModel(input_size, n_neurons, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Prepare DataLoaders
train_dataset = TensorDataset(torch.tensor(x_train_lstm, dtype=torch.float32),
                              torch.tensor(y_train_lstm, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(x_val_lstm, dtype=torch.float32),
                            torch.tensor(y_val_lstm, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training loop with early stopping
best_val_loss = np.inf
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_x.size(0)
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            val_loss += loss.item() * batch_x.size(0)
    val_loss /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    # Check early stopping condition
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), os.path.join(model_dir, 'DAP_Model_TLF.pt'))
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping")
            break

Epoch 1/50, Train Loss: 0.9614, Val Loss: 0.6701
Epoch 2/50, Train Loss: 0.8627, Val Loss: 0.4911
Epoch 3/50, Train Loss: 0.6881, Val Loss: 0.4161
Epoch 4/50, Train Loss: 0.5372, Val Loss: 0.4585
Epoch 5/50, Train Loss: 0.3990, Val Loss: 0.5118
Epoch 6/50, Train Loss: 0.2571, Val Loss: 0.3938
Epoch 7/50, Train Loss: 0.2280, Val Loss: 0.6567
Epoch 8/50, Train Loss: 0.2710, Val Loss: 0.4263
Epoch 9/50, Train Loss: 0.2786, Val Loss: 0.3696
Epoch 10/50, Train Loss: 0.2303, Val Loss: 0.3884
Epoch 11/50, Train Loss: 0.2157, Val Loss: 0.3615
Epoch 12/50, Train Loss: 0.1930, Val Loss: 0.3780
Epoch 13/50, Train Loss: 0.2003, Val Loss: 0.3908
Epoch 14/50, Train Loss: 0.2157, Val Loss: 0.4184
Epoch 15/50, Train Loss: 0.2306, Val Loss: 0.4119
Epoch 16/50, Train Loss: 0.1973, Val Loss: 0.4061
Epoch 17/50, Train Loss: 0.1849, Val Loss: 0.4129
Epoch 18/50, Train Loss: 0.1674, Val Loss: 0.3748
Epoch 19/50, Train Loss: 0.1780, Val Loss: 0.6190
Epoch 20/50, Train Loss: 0.2563, Val Loss: 0.4403
Epoch 21/

In [14]:
# Reload the best model
model.load_state_dict(torch.load(os.path.join(model_dir, 'DAP_Model_TLF.pt')))

# Make predictions on the test set
model.eval()
test_dataset = TensorDataset(torch.tensor(x_test_lstm, dtype=torch.float32),
                              torch.tensor(y_test_lstm, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

y_test_pred = []
with torch.no_grad():
    for batch_x, _ in test_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        y_test_pred.append(outputs.cpu().numpy())
y_test_pred = np.concatenate(y_test_pred, axis=0)

# Define an evaluation function
def evaluate_prediction(predictions, actual):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    return mae, rmse

print("Evaluation on normalized scale:")
evaluate_prediction(y_test_pred, y_test_lstm)

# Rescale predictions back to the original scale (using your stored standardization parameters)
y_test_pred_rescale = y_test_pred * y_std_reg.values + y_mean_reg.values
y_test_lstm_rescale = y_test_lstm * y_std_reg.values + y_mean_reg.values

# Inverse log transformation to get actual values
y_test_pred_invlog = np.exp(y_test_pred_rescale) - 1 + min(df["SCED"])
y_test_lstm_invlog = np.exp(y_test_lstm_rescale) - 1 + min(df["SCED"])

print("Evaluation after rescaling and inverse log:")
evaluate_prediction(y_test_pred_invlog, y_test_lstm_invlog)

Evaluation on normalized scale:
MAE: 0.35
RMSE: 0.73
Evaluation after rescaling and inverse log:
MAE: 34.38
RMSE: 190.66


(np.float64(34.37810028793424), np.float64(190.66390345059366))

In [18]:
# Ensure your model is in evaluation mode
model.eval()

# Compute baseline predictions and MSE on the validation set
with torch.no_grad():
    x_val_tensor = torch.tensor(x_val_lstm, dtype=torch.float32, device=device)
    baseline_pred_tensor = model(x_val_tensor)
    baseline_pred = baseline_pred_tensor.cpu().numpy()

baseline_mse = np.mean((baseline_pred - y_val_lstm)**2)
print("Baseline MSE:", baseline_mse)

# Initialize a dictionary to store importance values for each feature
feature_importance = {}
num_features = x_val_lstm.shape[2]
feature_names = ['DAP', 'SCED', 'F_TLF']  # adjust if needed

# Loop over each feature and measure the impact of shuffling its values
for feature in range(num_features):
    # Copy the validation data
    x_val_permuted = x_val_lstm.copy()
    # Shuffle the current feature's values for each time step across samples
    for t in range(x_val_lstm.shape[1]):
        x_val_permuted[:, t, feature] = np.random.permutation(x_val_permuted[:, t, feature])
    
    # Convert the permuted data to a torch tensor and get predictions
    with torch.no_grad():
        x_val_permuted_tensor = torch.tensor(x_val_permuted, dtype=torch.float32, device=device)
        permuted_pred_tensor = model(x_val_permuted_tensor)
        permuted_pred = permuted_pred_tensor.cpu().numpy()
    
    permuted_mse = np.mean((permuted_pred - y_val_lstm)**2)
    # Importance is the increase in MSE due to permutation
    importance = permuted_mse - baseline_mse
    feature_importance[feature_names[feature]] = importance

print("Feature Importance (increase in MSE):")
for feature, imp in feature_importance.items():
    print(f"{feature}: {imp:.4f}")

Baseline MSE: 0.34118851786140186
Feature Importance (increase in MSE):
DAP: 0.3272
SCED: 0.0317
F_TLF: 0.0450
